In [9]:
import sys
sys.path.append('/Users/gajendersharma/meepc')
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [10]:
hankel = Hankel()
corrhankel = Corrhankel() 
lag = 50
stride = 0.5

In [11]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [12]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME']]

In [13]:
y_truth=train_normal.loc[:,'ATT_FLAG']
y_truth


0        0
1        0
2        0
3        0
4        0
        ..
12933    0
12934    0
12935    0
12936    0
12937    0
Name: ATT_FLAG, Length: 12938, dtype: int64

In [14]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([418, 419, 420, 421, 430, 431, 432, 442, 443, 444, 445, 446, 462,
       463, 464, 465, 466, 467, 489, 490, 491, 492, 498, 499, 500, 501,
       502, 503, 506, 507, 508, 509, 510, 511])

In [15]:
train_normal



,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12933,24/12/16 20,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,0
12934,24/12/16 21,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,0
12935,24/12/16 22,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,0
12936,24/12/16 23,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,0


In [19]:
train_attack_only=train_normal[train_normal.ATT_FLAG==1]
train_attack_only

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
10488,13/09/16 23,3.51,2.55,3.60,2.24,1.62,4.97,2.84,93.19,1,...,86.28,22.50,82.83,16.84,79.83,16.56,80.37,39.76,24.20,1
10489,14/09/16 00,3.66,1.68,4.00,2.23,1.98,5.23,3.13,91.44,1,...,87.84,21.86,83.52,29.21,85.83,29.21,54.31,41.20,23.54,1
10490,14/09/16 01,3.84,0.83,4.44,2.46,2.51,5.47,3.45,90.22,1,...,87.25,21.13,84.47,29.89,86.62,29.89,55.20,41.96,22.79,1
10491,14/09/16 02,3.81,0.65,4.89,2.93,3.07,5.50,3.95,94.77,1,...,91.25,25.83,86.16,28.10,86.30,28.10,57.49,28.88,27.98,1
10492,14/09/16 03,3.75,1.00,5.24,3.42,3.67,5.36,4.78,94.80,1,...,76.13,27.65,88.04,18.20,83.38,17.95,85.43,30.34,29.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12793,19/12/16 00,0.39,3.81,4.02,3.28,4.00,5.50,3.77,98.55,1,...,89.52,28.10,86.71,19.27,82.66,19.16,70.54,30.92,30.02,1
12794,19/12/16 01,0.50,3.69,4.50,3.72,3.31,5.50,4.27,97.42,1,...,89.62,28.09,85.77,26.08,62.67,25.97,73.59,30.95,30.05,1
12795,19/12/16 02,0.66,3.61,4.97,3.90,2.61,5.50,4.95,96.87,1,...,92.70,28.10,87.05,29.10,62.90,29.10,60.38,30.96,30.06,1
12796,19/12/16 03,0.92,3.68,5.33,4.31,2.08,5.22,3.65,95.94,1,...,76.64,29.24,89.16,29.62,62.82,29.62,59.34,31.80,30.90,1


In [23]:
train_normal_only=train_normal[train_normal.ATT_FLAG==0]
train_normal_only

,DATETIME,L_T1,L_T2,L_T3,L_T4,L_T5,L_T6,L_T7,F_PU1,S_PU1,...,P_J256,P_J289,P_J415,P_J302,P_J306,P_J307,P_J317,P_J14,P_J422,ATT_FLAG
0,06/01/14 00,0.509730,2.049003,3.191145,2.792634,2.656091,5.316831,1.562321,98.998444,1,...,87.605774,26.495605,84.206619,18.901676,81.983734,18.791777,67.125603,29.387470,28.487471,0
1,06/01/14 01,0.412580,2.009072,3.642565,2.831673,3.126387,5.494855,1.852043,99.095901,1,...,89.448341,26.487326,85.900085,18.849329,82.150589,18.739643,67.178696,29.354256,28.454256,0
2,06/01/14 02,0.320112,1.986093,4.140192,3.256733,3.574601,5.500000,2.246126,98.420959,1,...,91.056114,26.487364,86.582474,19.597170,83.988579,19.496712,72.425293,29.354538,28.454538,0
3,06/01/14 03,0.332879,2.009203,4.673478,3.744497,3.952379,5.500000,3.203573,97.575172,1,...,92.594353,26.575815,88.020546,26.028486,64.670486,25.922703,76.275040,29.449951,28.549952,0
4,06/01/14 04,0.483496,2.089049,5.237937,4.409456,3.504676,5.500000,4.439714,97.351059,1,...,94.473099,26.723457,90.422462,26.209970,64.746620,26.104692,76.703529,29.574265,28.674263,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12933,24/12/16 20,2.650000,2.370000,3.850000,3.040000,3.820000,4.940000,2.190000,120.080000,1,...,70.030000,27.380000,84.140000,18.450000,81.670000,18.340000,66.040000,29.880000,28.980000,0
12934,24/12/16 21,2.240000,2.560000,3.420000,2.920000,3.690000,5.020000,1.970000,119.120000,1,...,68.600000,27.660000,83.460000,25.400000,60.850000,25.280000,66.890000,30.190000,29.290000,0
12935,24/12/16 22,1.910000,2.760000,2.950000,2.490000,2.700000,5.140000,1.870000,120.710000,1,...,85.630000,26.840000,82.820000,24.460000,59.560000,24.340000,66.080000,29.680000,28.780000,0
12936,24/12/16 23,1.520000,2.520000,3.330000,2.030000,1.690000,5.100000,1.390000,120.020000,1,...,86.150000,25.780000,103.630000,24.770000,59.010000,24.650000,66.420000,28.980000,28.080000,0


In [21]:
cluster_sensors = ['P_J302','S_PU1','F_PU3','S_PU3','F_PU5','S_PU5']

In [22]:
train_attack_only.loc[:,cluster_sensors]

,P_J302,S_PU1,F_PU3,S_PU3,F_PU5,S_PU5
10488,16.84,1,0,0,0,0
10489,29.21,1,0,0,0,0
10490,29.89,1,0,0,0,0
10491,28.10,1,0,0,0,0
10492,18.20,1,0,0,0,0
...,...,...,...,...,...,...
12793,19.27,1,0,0,0,0
12794,26.08,1,0,0,0,0
12795,29.10,1,0,0,0,0
12796,29.62,1,0,0,0,0


In [31]:
#temp
li=[4,5,6,7]
a=np.ones(1)
li[a]

TypeError: only integer scalar arrays can be converted to a scalar index

In [24]:
train_normal_only.loc[:,cluster_sensors]

,P_J302,S_PU1,F_PU3,S_PU3,F_PU5,S_PU5
0,18.901676,1,0,0,0,0
1,18.849329,1,0,0,0,0
2,19.597170,1,0,0,0,0
3,26.028486,1,0,0,0,0
4,26.209970,1,0,0,0,0
...,...,...,...,...,...,...
12933,18.450000,1,0,0,0,0
12934,25.400000,1,0,0,0,0
12935,24.460000,1,0,0,0,0
12936,24.770000,1,0,0,0,0


In [25]:
meepc_sensors = ['P_J317','S_PU2','F_PU2','F_PU8','P_J269']

In [26]:
train_attack_only.loc[:,meepc_sensors]

,P_J317,S_PU2,F_PU2,F_PU8,P_J269
10488,80.37,1,93.21,34.46,36.62
10489,54.31,1,91.46,37.93,37.54
10490,55.20,1,90.24,37.87,38.18
10491,57.49,1,94.79,37.08,35.78
10492,85.43,1,94.81,33.13,35.76
...,...,...,...,...,...
12793,70.54,1,98.57,34.16,33.75
12794,73.59,1,97.44,0.00,34.36
12795,60.38,1,96.89,0.00,34.66
12796,59.34,1,95.96,0.00,35.15
